# Helmet Detection - CenterNet ResNet50 V2 512x512:

## 1. DOWNLOAD THE PRETRAINED MODELS FROM THE TENSOFLOW MODEL ZOO

In [1]:
import os

# DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join('pre-trained-model')
for dir in [MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [2]:
main_directory = 'models'
subfolder_name = 'centernet_resnet50_v2_512x512' #Change

main_directory_path = os.path.join(os.getcwd(), main_directory)
subfolder_path = os.path.join(main_directory_path, subfolder_name)

if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)

In [3]:
import tarfile
import urllib.request
import os

# Download and extract model
MODEL_DATE = '20200711' #Change
MODEL_NAME = 'centernet_resnet50_v2_512x512_coco17_tpu-8' #Change
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

## 2. SETUP PATHS:

In [4]:
WORKSPACE_PATH = '../training_helper_directory'
SCRIPT_PATH = WORKSPACE_PATH + '/scripts'
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'

IMAGE_PATH = '../Dataset/"PASCAL VOC Format Dataset"'

CUSTOM_MODEL_NAME = '/centernet_resnet50_v2_512x512' #Change: same as subfolder name
MODEL_PATH = './models' + CUSTOM_MODEL_NAME

PRETRAINED_PATH = './pre-trained-model/' + MODEL_NAME
CONFIG_PATH = MODEL_PATH + '/pipeline.config'

EXPORT_PATH = '../exported-models/my-model'

## 3. CREATE TF RECORDS:

In [5]:
# Create train data:
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}

# Create test data:
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: ../training_helper_directory/annotations/train.record
Successfully created the TFRecord file: ../training_helper_directory/annotations/test.record


## 4. COPY MODEL CONFIG FILE TO TRANING FOLDER:

In [6]:
SOURCE_PATH = "D:\Helmet Detection System\CenterNet ResNet50 V2 512x512\pre-trained-model\centernet_resnet50_v2_512x512_coco17_tpu-8\pipeline.config" #Change
DESTINATION_PATH = "D:\Helmet Detection System\CenterNet ResNet50 V2 512x512\models\centernet_resnet50_v2_512x512" #Change

In [7]:
!copy "{SOURCE_PATH}" "{DESTINATION_PATH}"

        1 file(s) copied.


## 5. UPDATING CONFIG FOR TRANSFER LEARNING:

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [10]:
config

{'model': center_net {
   num_classes: 90
   feature_extractor {
     type: "resnet_v2_50"
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 512
       max_dimension: 512
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 },
 'train_config': batch_size: 128
 data_augmentation_options {
   random_horizontal_flip {
   }
 }
 data_augmentation_options {
   random_crop_image {
     min_aspect_ratio: 0.5
     max_aspect_ratio: 1.7000000476837158
     random_coef: 0.25
   }
 }
 data_augmentation_options {


In [11]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config.model.center_net.num_classes = 2 #Change num_classes and name

pipeline_config.train_config.batch_size = 2 #Change
pipeline_config.train_config.num_steps = 30000 #Change

pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_PATH + '/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection" 

pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH+ '/train.record'] 

pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

## 6. TRAIN THE MODEL:

In [14]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
import os
os.getcwd()

'd:\\Helmet Detection System\\CenterNet ResNet50 V2 512x512'

In [16]:
# !python {SCRIPT_PATH + '/model_main_tf2.py'} --model_dir={MODEL_PATH} --pipeline_config_path={MODEL_PATH + '/pipeline.config'}

In [17]:
# !python {SCRIPT_PATH + '/exporter_main_v2.py'} --input_type image_tensor --pipeline_config_path {CONFIG_PATH} --trained_checkpoint_dir {MODEL_PATH + '/' } --output_directory exported-models\my-model

Inferencing My Trained Models

In [18]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
# from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = 'D:/Helmet Detection System/Dataset/PASCAL VOC Format Dataset/test/BikesHelmets392_png.rf.9802b10c090e222ccda1a9c575875961.jpg' #Change


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = './exported-models/my-model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=100,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
cv2.imshow('test', image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading model...Done! Took 6.428758382797241 seconds
Running inference for D:/Helmet Detection System/Dataset/PASCAL VOC Format Dataset/test/BikesHelmets392_png.rf.9802b10c090e222ccda1a9c575875961.jpg... Done


In [19]:
# !python {SCRIPT_PATH + '/model_main_tf2.py'} --model_dir={MODEL_PATH} --pipeline_config_path={MODEL_PATH + '/pipeline.config'} --checkpoint_dir={MODEL_PATH}

In [1]:
# Suppress TensorFlow logging (optional)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pathlib
import tensorflow as tf
import cv2
import argparse

# Enable GPU dynamic memory allocation (optional)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = './exported-models/my-model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# Load saved model and build detection function
print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_MODEL_DIR + "/saved_model")

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# Load label map data for plotting
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS, use_display_name=True)

# Initialize webcam capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame from webcam
    ret, frame = cap.read()

    if not ret:
        print("Error capturing frame!")
        break

    # Resize and preprocess frame (example)
    image = cv2.resize(frame, (640, 640))  # Adjust to your model's input size
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Convert to tensor and add batch dimension
    input_tensor = tf.convert_to_tensor(image_rgb)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run inference
    detections = detect_fn(input_tensor)

    # Parse detections and draw results
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_with_detections = image.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        min_score_thresh=MIN_CONF_THRESH,
        agnostic_mode=False
    )

    # Display annotated frame
    cv2.imshow('Object Detection', image_with_detections)

    # Exit on 'q' key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

NameError: name 'os' is not defined